In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'Malgun Gothic' 
matplotlib.rcParams['font.size'] = 15 
matplotlib.rcParams['axes.unicode_minus'] = False
from tensorflow import keras
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier # kaggle에서 5년 연속 1위 븐류모델-> epoch돌때마다 손실값 뜸
from xgboost import plot_importance 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy.stats import loguniform, uniform, randint

In [28]:
df=pd.read_csv('./train.csv',index_col='id')
df

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
id,,,,,,,,
0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert
...,...,...,...,...,...,...,...,...
18519,3.0,No,7.0,3.0,No,9.0,7.0,Extrovert
18520,1.0,NaN,6.0,7.0,No,6.0,5.0,Extrovert
18521,7.0,Yes,1.0,1.0,Yes,1.0,NaN,Introvert


In [3]:
df.isnull().sum()

id                              0
Time_spent_Alone             1190
Stage_fear                   1893
Social_event_attendance      1180
Going_outside                1466
Drained_after_socializing    1149
Friends_circle_size          1054
Post_frequency               1264
Personality                     0
dtype: int64

In [9]:
df2=df.dropna(axis='index')

In [10]:
df2 #결측치 들어간 행: 8000행 

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
id,,,,,,,,
0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
7,2.0,No,8.0,3.0,No,4.0,5.0,Extrovert
9,1.0,No,8.0,6.0,No,14.0,9.0,Extrovert
...,...,...,...,...,...,...,...,...
18509,1.0,No,3.0,4.0,No,15.0,4.0,Extrovert
18511,0.0,No,4.0,5.0,No,11.0,4.0,Extrovert
18514,6.0,No,5.0,3.0,No,10.0,4.0,Extrovert


In [11]:
df[(df['Stage_fear'].isnull())&(df['Drained_after_socializing'].isnull())]

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
id,,,,,,,,
514,11.0,NaN,2.0,0.0,NaN,5.0,0.0,Introvert
890,10.0,NaN,3.0,2.0,NaN,4.0,0.0,Introvert
1336,7.0,NaN,0.0,2.0,NaN,3.0,2.0,Introvert
1440,1.0,NaN,9.0,5.0,NaN,13.0,9.0,Extrovert
1529,9.0,NaN,2.0,1.0,NaN,2.0,2.0,Introvert
2350,3.0,NaN,8.0,3.0,NaN,14.0,4.0,Extrovert
2809,0.0,NaN,9.0,2.0,NaN,5.0,2.0,Extrovert
3372,2.0,NaN,9.0,3.0,NaN,11.0,4.0,Extrovert
3606,9.0,NaN,1.0,1.0,NaN,2.0,2.0,Introvert


In [20]:
def ohe(x):
    return 1 if x=='Yes' else (0 if x=='No' else np.nan)

In [29]:
le=LabelEncoder()
xy=df
xy[['Stage_fear','Drained_after_socializing']]=df[['Stage_fear','Drained_after_socializing']].applymap(ohe)
xy['Personality']=le.fit_transform(xy['Personality'])  # extrovert=0, introvert=1
xy

C:\Users\itwill\AppData\Local\Temp\ipykernel_7360\2068477150.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  xy[['Stage_fear','Drained_after_socializing']]=df[['Stage_fear','Drained_after_socializing']].applymap(ohe)


,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
id,,,,,,,,
0,0.0,0.0,6.0,4.0,0.0,15.0,5.0,0
1,1.0,0.0,7.0,3.0,0.0,10.0,8.0,0
2,6.0,1.0,1.0,0.0,NaN,3.0,0.0,1
3,3.0,0.0,7.0,3.0,0.0,11.0,5.0,0
4,1.0,0.0,4.0,4.0,0.0,13.0,NaN,0
...,...,...,...,...,...,...,...,...
18519,3.0,0.0,7.0,3.0,0.0,9.0,7.0,0
18520,1.0,NaN,6.0,7.0,0.0,6.0,5.0,0
18521,7.0,1.0,1.0,1.0,1.0,1.0,NaN,1


In [37]:
print(np.unique(df['Personality'],return_counts=True))
class_weights = {
    0: 18524 / (2 * 13699),  # ≈ 0.676
    1: 18524 / (2 * 4825)    # ≈ 1.92
}

(array([0, 1]), array([13699,  4825], dtype=int64))


In [34]:
x=xy.iloc[:,:-1]; y=xy.iloc[:,-1]
x_train,x_test,y_train, y_test= train_test_split(x,y,test_size=0.2,random_state=123)

In [41]:
            params = {'colsample_bytree': uniform(0.5,1), #각 트리 생성 시 사용하는 feature 비율
                      'learning_rate': uniform(0.01, 0.3), #값이 낮을수록 학습이 느리지만 일반화 성능 좋음
                      'max_depth' : randint(5,15),
                      'min_child_weight' : randint(1, 5), #자식 노드 분할을 결정하는 최소 가중치의 합 -> 작으면 더 만흥 자식 노드 분할
                      'n_estimators' : [100, 200, 300,500]} # 트리개수

In [42]:
xc=XGBClassifier(objective='binary:logistic',eval_metric='logloss')
rs=RandomizedSearchCV(xc,param_distributions=params,n_iter=100,n_jobs=-1,random_state=123)
from sklearn.utils.class_weight import compute_sample_weight

# 클래스별 가중치 정의 (예: 반비례 방식)
class_weights = {0: 0.676, 1: 1.92}  # 예시

# 샘플별 가중치 생성
sample_weights = compute_sample_weight(class_weight=class_weights, y=y_train)

# RandomizedSearchCV에 넘기려면 fit_params를 통해 전달
rs.fit(x_train, y_train, **{'sample_weight': sample_weights})
rs.best_params_, np.max(rs.cv_results_['mean_test_score'])

C:\Users\itwill\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
270 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\itwill\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\itwill\anaconda3\Lib\site-packages\xgboost\core.py", line 729, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "C:\Users\itwill\anaconda3\Lib\site-packages\xgboost\sklearn.py", line 1682, in fit
    self._Booster = train(
                    ^^^^^^
  File "C:\Users\it

({'colsample_bytree': 0.741827618844835,
  'learning_rate': 0.02682490361648498,
  'max_depth': 6,
  'min_child_weight': 2,
  'n_estimators': 100},
 nan)

In [43]:
model=rs.best_estimator_

In [44]:
model.score(x_test,y_test)

0.9686909581646423

In [45]:
model.score(x_train,y_train)

0.9703758688170592

In [48]:
test=pd.read_csv('./test.csv',index_col='id')
test

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
id,,,,,,,
18524,3.0,No,7.0,4.0,No,6.0,NaN
18525,NaN,Yes,0.0,0.0,Yes,5.0,1.0
18526,3.0,No,5.0,6.0,No,15.0,9.0
18527,3.0,No,4.0,4.0,No,5.0,6.0
18528,9.0,Yes,1.0,2.0,Yes,1.0,1.0
...,...,...,...,...,...,...,...
24694,3.0,No,5.0,5.0,No,9.0,6.0
24695,8.0,Yes,2.0,1.0,Yes,0.0,0.0
24696,2.0,No,4.0,3.0,No,9.0,7.0


In [49]:
test[['Stage_fear','Drained_after_socializing']]=test[['Stage_fear','Drained_after_socializing']].applymap(ohe)
test

C:\Users\itwill\AppData\Local\Temp\ipykernel_7360\3627977440.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test[['Stage_fear','Drained_after_socializing']]=test[['Stage_fear','Drained_after_socializing']].applymap(ohe)


,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
id,,,,,,,
18524,3.0,0.0,7.0,4.0,0.0,6.0,NaN
18525,NaN,1.0,0.0,0.0,1.0,5.0,1.0
18526,3.0,0.0,5.0,6.0,0.0,15.0,9.0
18527,3.0,0.0,4.0,4.0,0.0,5.0,6.0
18528,9.0,1.0,1.0,2.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...
24694,3.0,0.0,5.0,5.0,0.0,9.0,6.0
24695,8.0,1.0,2.0,1.0,1.0,0.0,0.0
24696,2.0,0.0,4.0,3.0,0.0,9.0,7.0


In [50]:
y_pred=model.predict(test)

In [53]:
y_pred=le.inverse_transform(y_pred)

In [54]:
y_pred

array(['Extrovert', 'Introvert', 'Extrovert', ..., 'Extrovert',
       'Extrovert', 'Introvert'], dtype=object)

In [56]:
test_ids=test.index

In [58]:
submission = pd.DataFrame({
    'id': test_ids,           # 보통 test.csv에서 가져온 ID 열
    'Personality': y_pred           # 모델이 예측한 결과
})

submission.to_csv('submission.csv', index=False)